In [1]:
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import make_data
import datetime
import requests
import glob
import sys
import os
import json
import time
import re

In [2]:
names = [405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 456, 419, 420, 421, 422, 838, 839, 840, 841, 842, 843, 884, 423, 424, 425, 426, 427, 1497, 428, 429, 430, 997, 1028, 1029, 1030, 1031, 1046, 1144, 1121, 1127, 1261, 1262, 1334, 1430]

In [3]:
len(names)

48

In [4]:
sm = 0
for i in names:
    f = open('data/categories/{}_urls.txt'.format(i))
    lines = f.readlines()
    sm += len(lines)
    f.close()
print(sm)

7238


In [5]:
tree = np.load('data/tree.npy')

In [6]:
def dfs(tree, parents, childrens):
    mas = []
    if tree['children'] != None:
        for i in tree['children']:
            parents[i['a_attr']['title'].split('.')[0]] = tree['a_attr']['title'].split('.')[0]
            mas.extend([i['a_attr']['title'].split('.')[0]])
            dfs(i, parents, childrens)
    childrens[tree['a_attr']['title'].split('.')[0]] = mas

In [7]:
parents = {'0':'0'}
childrens = {}
for i in tree:
    parents[i['a_attr']['title'].split('.')[0]] = '0'
    dfs(i, parents, childrens)

In [8]:
def getData(tree, names, allUrls, parent_numb, st):
    dataUrls = []
    dataText = []
    dataCategories = []
    
    number = int(tree['a_attr']['title'].split('.')[0])
    if number in names and number in files:
        f = open('data/filterData/categories/{}.txt'.format(number))
        lines = f.readlines()
        categUrls = [i.split(', ', 1)[0] for i in lines]
        categText = [i.split(', ', 1)[1] for i in lines]
        f.close()
        
        for i in range(len(categUrls)):
            if categUrls[i] not in allUrls:
                dataUrls.extend([categUrls[i]])
                dataText.append([categText[i].replace('\n', '')])
                dataCategories.extend([number])

                allUrls.add(categUrls[i])
            
    if len(tree['children']) != 0:
        for i in tree['children']:
            a, b, c = getData(i, names, allUrls, number, st)
            dataUrls.extend(a)
            dataText.extend(b)
            dataCategories.extend(c)
    
    else:
        if number in names and number in files \
            and (len(categUrls) - len(dataUrls)) / len(categUrls) > 0.8:
            dataCategories = (np.ones(len(dataCategories)) * parent_numb).tolist()

    return dataUrls, dataText, dataCategories

In [9]:
files = glob.glob('data/filterData/categories/*')
files = [int(i[27:].split('.')[0]) for i in files]

In [10]:
allUrls = set()
dataUrls = []
dataText = []
dataCategories = []
st = '   '
parent_numb = 1
for i in tree:
    a, b, c = getData(i, names, allUrls, parent_numb, st)
    dataUrls.extend(a)
    dataText.extend(b)
    dataCategories.extend(c)

In [11]:
dataUrls = np.array(dataUrls)
dataText = np.array(dataText,  dtype='O')
dataCategories = np.array(dataCategories)

In [12]:
len(set(dataCategories))

47

In [13]:
len(dataUrls)

6255

In [14]:
dataAns = [(dataUrls[i], dataCategories[i]) for i in range(dataUrls.shape[0])]

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
trainText, testText, trainAns, testAns = \
    train_test_split(dataText, dataAns, test_size=0.2)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestCentroid, KNeighborsClassifier, NearestNeighbors
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.linear_model import RidgeClassifier, LinearRegression, LogisticRegression

/home/adil/.pyenv/versions/3.6.1/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
trainText = np.load('data/filterData/patTrainText.npy')
trainAns = np.load('data/filterData/patTrainAns.npy')
testText = np.load('data/filterData/patTestText.npy')
testAns = np.load('data/filterData/patTestAns.npy')

In [19]:
f = open('fasttext/train.txt', 'w')
for i in range(trainText.shape[0]):
    f.writelines('__label__'+trainAns[i][1]+' '+trainText[i]+'\n')
f.close()

f = open('fasttext/testA.txt', 'w')
for i in range(testText.shape[0]):
    f.writelines('__label__'+testAns[i][1]+' '+testText[i]+'\n')
f.close()

f = open('fasttext/testN.txt', 'w')
for i in range(testText.shape[0]):
    f.writelines(testText[i]+'\n')
f.close()

In [20]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

In [22]:
f = open('fasttext/ans.txt')
lines = f.readlines()
predict = np.array([float(i.replace('__label__', '').replace('\n', '')) for i in lines])

In [23]:
sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

0.701838529177
0.7897681854516387


In [21]:
import os
os.remove('fasttext/ans.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'fasttext/ans.txt'

In [49]:
trainCategory = [float(i[1]) for i in trainAns]
testCategory = [float(i[1]) for i in testAns]

In [16]:
trainTexts = trainText
testTexts = testText

In [19]:
trainTexts = [i[0] for i in trainText]
testTexts = [i[0] for i in testText]

### tf-idf

In [37]:
tfidfTransformer = TfidfVectorizer(ngram_range=(1,3))
trainFeatures  = tfidfTransformer.fit_transform(trainTexts)
testFeatures = tfidfTransformer.transform(testTexts)

In [38]:
trainFeatures

<5003x4368432 sparse matrix of type '<class 'numpy.float64'>'
	with 7253108 stored elements in Compressed Sparse Row format>

In [22]:
# from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# ngramTransformer = CountVectorizer(ngram_range=(3,3))
# trainFeatures = ngramTransformer.fit_transform(trainTexts)
# testFeatures = ngramTransformer.transform(testTexts)

# CLF

### 1,2

In [39]:
_time = time.time()
clf = LogisticRegression()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  353.3719129562378
0.678657074341
0.7474020783373302


In [22]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  196.75042819976807
0.749000799361
0.8225419664268585


In [23]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  1.2476506233215332
0.673061550759
0.8353317346123101


In [24]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.024344205856323242
0.639488409273
0.7242206235011991


In [25]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.02592611312866211
0.674660271783
0.768185451638689


In [26]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  124.82207012176514
0.116706634692
0.1486810551558753


In [27]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  221.21249890327454
0.686650679456
0.7697841726618705


### 2,2

In [30]:
_time = time.time()
clf = LogisticRegression()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  131.48263025283813
0.482813749001
0.5315747402078337


In [31]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  157.82303643226624
0.687450039968
0.7601918465227818


In [32]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  1.7989120483398438
0.585131894484
0.7442046362909672


In [33]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.023096323013305664
0.109512390088
0.31095123900879296


In [34]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.020958900451660156
0.585131894484
0.6994404476418865


In [35]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  118.1530339717865
0.116706634692
0.1486810551558753


In [36]:
_time = time.time()
clf = SVC(gamma=0.5)
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  279.654146194458
0.426059152678
0.4660271782573941


### 3,3

In [42]:
_time = time.time()
clf = RidgeClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  3995.5270805358887
0.592326139089
0.6546762589928058


In [51]:
_time = time.time()
clf = NearestCentroid()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  1.5285096168518066
0.220623501199
0.2709832134292566


In [52]:
_time = time.time()
clf = KNeighborsClassifier()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.04726719856262207
0.139888089528
0.4844124700239808


In [53]:
_time = time.time()
clf = KNeighborsClassifier(algorithm='brute', metric='cosine')
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  0.013119220733642578
0.494004796163
0.6091127098321343


In [54]:
_time = time.time()
clf = SVC()
clf.fit(trainFeatures, trainCategory)
print('Fit time: ', time.time() - _time)
predict = clf.predict(testFeatures)

sm = 0
print(accuracy_score(testCategory, predict))
for i in range(len(testCategory)):
    if testCategory[i] == predict[i] or \
        parents[str(int(testCategory[i]))] == str(int(predict[i])) or\
        str(int(predict[i])) in childrens[str(int(testCategory[i]))]:
        sm += 1
print(sm / len(testCategory))

Fit time:  114.99397683143616
0.113509192646
0.14948041566746603
